# crawler tutorial

## Content

这篇文章将主要讲述 python 爬虫的基本用法以及实操建议，主要内容包括：

• 最基本的 HTTP 知识以及如何利用浏览器 inspect 功能

• 爬取网页的两大方式：利用 requests 发送请求，以及利用 selenium 模拟浏览器访问网页

• 实际操作过程中的注意事项

• 以不同类型的网页为例讲解什么情况下具体应当使用哪种方式

## 环境准备

运行本教程中的代码需要安装如下库: requests, selenium, fake_useragent, webdriver_manager

```shell
pip3 install requests
pip3 install selenium
pip3 install fake_useragent
pip3 install webdriver_manager
```

# 一、前置知识

在爬取网页之前，首先需要简单了解 HTTP 基本知识，以及浏览器 inspect 功能的用法


## HTTP基本知识



### 1.什么是 HTTP

HTTP:Hypertext Transfer Protocol（超文本传输协议）是客户端与服务端之间的一个简单的请求-响应协议。

Example:

客户端（浏览器）向服务端发送了一个 HTTP 请求(比如 get )，服务端则返回客户端一个响应(response)。响应中包含了响应的状态信息，并且可能携带了所请求的内容。


### 2.HTTP 方法

最常用的 HTTP 方法为 get 和 post

爬虫中需要用到的是 get 方法，get 方法即是向特定的信息源请求数据.

而 post 则是向一个特定的服务端发送数据


### 3.HTTP 响应状态码

HTTP 响应状态码被用于表明特定的HTTP请求是否完成/错误原因。响应有以下5类：

• 信息响应 (100–199)

• 成功响应 (200–299)

• 重定向消息 (300–399)

• 客户端错误响应 (400–499)

• 服务端错误响应 (500–599)

值得注意的响应状态码有：

200: 响应成功，爬取网页时，get 请求的 status code 为200，即请求成功。但注意请求成功不代表正确返回所需内容，返回的也有可能是验证码界面等。

404: not found. 该状态码表明服务器上无法找到请求的资源。除此之外，也可以在服务器端拒绝请求且不想说明理由时使用。

429: too many requests. 用户在给定的时间内发送了太多请求（"限制请求速率"）。这也是常见的反爬手段之一，如果频繁向 google search 请求信息，谷歌很快会开始返回429。所以，爬取的过程中注意 sleep ，将程序停止一段时间。

更详细的 status code 信息请参见：https://developer.mozilla.org/zh-CN/docs/Web/HTTP/Status


### 4.Cookie

HTTP Cookie是服务器发送到用户浏览器并保存在本地的一小块数据，它会在浏览器下次向同一服务器再发起请求时被携带并发送到服务器上。通常，它用于告知服务端两个请求是否来自同一浏览器，如保持用户的登录状态。

Cookie 主要用于以下三个方面：

- 会话状态管理（如用户登录状态、购物车、游戏分数或其它需要记录的信息）
- 个性化设置（如用户自定义设置、主题等）
- 浏览器行为跟踪（如跟踪分析用户行为等）

爬取网页时，有些网页需要登录才能看到具体内容。这种情况下get请求需要传入cookie来表明你登录了


### 5.User-Agent

User-Agent 即用户代理，简称“UA”，它是一个特殊字符串头。网站服务器通过识别 “UA”来确定用户所使用的操作系统版本、CPU 类型、浏览器版本等信息。而网站服务器则通过判断 UA 来给客户端发送不同的页面。

比如 Mac 上 Chrome 的 User-Agent 就是：`Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36`

爬虫使用代码来访问网站，其在进行get请求时可能带有明显标识为网络爬虫的 user-agent 信息。网站可以通过识别请求头中 user-agent 信息来判断是否是爬虫访问网站。如果是，网站首先对该 IP 进行预警，对其进行重点监控，当发现该 IP 超过规定时间内的访问次数，将在一段时间内禁止其再次访问网站。

Cookie 和 User-Agent 都包含在 http 请求的 headers 请求头中。



## 利用浏览器的开发者工具



### 1.利用浏览器开发者工具获取请求的信息

以 chrome 浏览器为例。

开发者工具打开方式：在 Chrome 界面按 F12 或者在页面上右键点击，选择“检查”(inspect)，就会出现开发者工具。

这里讲解利用开发者工具最上栏的 network 查找 Requested URL 以及请求的 headers信息。

比如你希望找到某 up 主有多少人关注。于是你进入了他的主页 `https://space.bilibili.com/10462362?spm_id_from=333.337.0.0`。

打开 network 后刷新页面，network 中的 overview 栏就出现了许多横线。如果横线出现的次序先后明显的话，是可以看出对应关系的，比如 google 搜索就比较明显。

但是在个情境下，你可能没办法直接看出究竟是哪个请求返回了关注数量，所以你先可以先从主页中看出该 up 主大约有2272.6万人关注，然后在左边的 search 中搜索2272，就可以找到所想要查找的包了,点击就可以发现请求的 Request URL:`https://api.bilibili.com/x/space/upstat?mid=10462362&jsonp=jsonp`。

以及向下翻，可以在 Request Headers 下看到自己的 cookie 和 user-agent。

### 2.利用浏览器开发者工具检查元素

你想知道该up主的主页中，'关注'按钮在html中的具体信息，可以直接右击关注，选择 inspect（这里可能需要重复操作两次），就可以看到其信息：`<span class="h-f-btn h-follow">`

# 二、爬取网页的两大方式

爬取网页的两大方式为 requests 与 selenium。其中 requests 为直接在代码中请求数据，而 selenium 则会模拟浏览器访问页面
本部分将以爬取不同网站作为例子来讲解两种方式。


## requests

本部分将主要讲解 requests.get() 方法，以及为了请求到想要的数据所需传入的参数，包括 headers(User-Agent & cookie) 以及 proxies 参数

使用requests需要先import requests库

In [1]:
import requests

### 1. requests.get

最基本地，利用 `requests.get(URL)` 即可以请求所需要的信息。`requests.get()` 即是通过上述 HTTP get 方法来向服务端请求信息

比如，我们试着请求天涯杂谈的首页:

In [2]:
resp = requests.get("http://bbs.tianya.cn/list-free-1.shtml")

我们可以查看一下这次请求所返回的网页源码

In [ ]:
resp.text

恭喜你！这样你就成功完成了一个请求！如果你的任务是从该页面中提取信息，那么爬取的任务就这么简单地完成了。接下来就是要做的就是通过beautifulSoup等方法解析这个网页了。


### 2. user-agent

但是，有的网页直接利用 get 会失败，比直接如访问豆瓣。

In [3]:
resp = requests.get("https://www.douban.com/")

In [4]:
resp

<Response [418]>

没有能正确返回所请求的数据。

我们再来看看 resp 的 headers, 可以发现其 user-agent 明显为爬虫头：python-requests/2.28.1

In [5]:
resp.request.headers

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

我们在上面讲过，网页可能通过 user-agent 和 cookie 来判断来访者是不是网络爬虫，这里就是因为 get 请求没有带上正常的 user-agent 导致请求被拒绝，所以我们可以通过上述方法利用浏览器的 inspect 找到自己的 User-Agent，然后添加在 get 方法 headers 参数中的 "User-Agent" 元素里。

In [6]:
resp = requests.get("https://www.douban.com/",
                    headers={"user-agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})

In [7]:
resp

<Response [200]>

加上 user-agent ，豆瓣就能返回所需信息了。

user-agent 可以通过上述的 inspect 方法获取。但更好的方式是利用 fake_useragent 库，不仅能免去复制长串字符的烦恼，还能通过随机选择 user-agent 来防止网页拒绝来自同一个 user-agent 的大量请求。尝试重复运行下面的代码，可以得到不同的 user-agent 。

In [8]:
from fake_useragent import UserAgent as ua
print(ua(use_cache_server=False).random)

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


爬取的时候可以直接套用 `headers={ua(use_cache_server=False).random}` ，简单好用，何乐而不为

注： `use_cache_server=False` 为禁用浏览器缓存

### 3.cookie

上面我们讲过，网站可以通过cookie来判断用户是否登录，登录了才会返回更多的信息。所以，我们可以通过传入cookie来获取登录后的信息。cookie的获取方式同上所述。

比如 bilibili 只有登录了才会显示 up 主的 likes 以及 view。我们通过上述方法，利用 inspect 功能找到请求 likes 数据的包，得到了"https://api.bilibili.com/x/space/upstat?mid=13638568&jsonp=jsonp"。再通过requests.get()对数据进行请求。

In [9]:
resp = requests.get("https://api.bilibili.com/x/space/upstat?mid=10462362&jsonp=jsonp" , headers={
    "User-Agent": ua(use_cache_server=False).random})

In [10]:
resp.text

'{"code":0,"message":"0","ttl":1,"data":{}}'

可以发现 bilibili 并没有给我们返回所需要的信息。而如果你推出登录，再访问同样的界面，你会发现你也无法在网页上看到 up 主关注和点赞的具体信息。

但是如果带上了 cookie，就能正确显示信息了！请在下面的cookie变量中输入你自己的cookie

In [11]:
cookie = ""
headers = {"user-agent": ua(use_cache_server=False).random, "cookie": cookie}
resp = requests.get("https://api.bilibili.com/x/space/upstat?mid=10462362&jsonp=jsonp" , headers=headers)

In [12]:
resp.text

'{"code":0,"message":"0","ttl":1,"data":{"archive":{"view":1069784327},"article":{"view":14789932},"likes":22727471}}'


### 4.proxies

上面提到，如果反复对同一个网站发起请求，你的 IP 会被网页封禁。例如， google 会返回429，百度将返回验证码页。这种情况有的时候可以通过增大 time.sleep() 的时间，即程序中止的时间来解决，但有的时候即便 sleep 值很大仍然会发生各种各样的错误，所以可能需要 IP 池，并将 IP 信息写入 get 方法的 proxies 参数。使用 IP 池则可以不用 sleep。

proxies 的参数可以被这样传入：

In [ ]:
cookie = ""
ip = ""
port = ""
proxies={'http': f'{ip}:{port}'}
resp = requests.get("http://bbs.tianya.cn/list-free-1.shtml" , headers=headers, proxies=proxies)

IP 池的购买和使用请自行善用搜索。



## selenium

有的时候，网页会动态返回内容。比如，你翻页翻到下方才能自动加载更多的评论。这种情况下，直接用 requests.get() 只能获得网页的 javascript 源代码，所以我们可以通过 selenium 模拟用户操作来获取所需内容。

与利用 requests 时思考如何完善请求的信息不同，利用 selenium ，你需要做的事情就是思考如何才能更好模拟用户实际操作。

本部分将以使用 selenium 登录 WebVPN 为例子，讲解 selenium 的使用方法。涉及的内容包括：

• DriverManager 创建浏览器

• get 方法访问页面

• 找到元素并 send_keys 向搜索框/密码框中发送值

• WebDriverWait等待页面加载

• action_chain 自动按键

• switch_to.window() 切换所操作的界面

• page_source 获取动态加载后的网页源代码


### 1.建立chrome浏览器

以下的代码创建了一个新的 chrome 浏览器

其中，service 参数中 ChromeDriverManager().install() 在你每次打开浏览器的时候,会检查你的浏览器版本/驱动是不是最新的，如果不是，就自动下载下来。否则可能发生 webdriver 不支持你电脑中 Chrome 版本的错误。

In [21]:
# 用来创建 chrome 浏览器的三个库
import selenium
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

d = selenium.webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

### 2.访问目标网页

直接用 get() 方法即可以访问目标网页，在 selenium 中无需考虑 headers 等问题

比如我们访问 WebVPN 。运行下面的页面， chrome 自动跳转到 webVPN 的界面

In [14]:
d.get("https://webvpn.tsinghua.edu.cn/login")

### 3.定位网页元素

我们可以通过 d.find_element() 方法来查找网页中的元素，而 selenium.webdriver.common.by 提供给我们优雅的寻找元素的方式。

比如我们想查找 WebVPN 中 username 的位置，可以通过 inspect 先找到 username 栏对应的 html 代码位置。发现其 tag 是一个 input，id 是 userCode。故我们就可以通过以下的方式来查找：

In [15]:
from selenium.webdriver.common.by import By # 搜寻元素
username = d.find_element(By.ID, "user_name")

By 提供的查找元素的方式有通过 ID, TAG_NAME, NAME, CLASS_NAME 等。但是通过这些方法查找，你需要保证你所查找的元素是唯一的，或者你需要知道它是满足同样条件的元素中第几个元素，这有时会带来不便。最万金油的方法是通过 `By.XPATH` 。可能语法看起来麻烦，但有的时候会很方便。

这里简单以 username 为例讲解 XPATH。 username 的XPATH为 '//input[@id="userCode"]'。

'//'后的元素为 tagname, '[]'中为元素的 attribute 以及其值（ username 的 id attribute 值即为 userCode ）。

当然了，你还可以这样定位 username: '//input[@id="userCode" and @type="text"]'

上面的这么多已经够用了。想了解更多有关 XPATH 的知识可以参见：`https://www.w3schools.com/xml/xpath_intro.asp`。

找到了 username 之后，我们就可以方便地使用 send_keys method 来向其中输入值。

In [16]:
username.send_keys("2021000000") # plz fill in your own username

请你通过同样的方法找到 password 和登录按键 btn。并通过 btn.click() 登入 webVPN。

### 4.等待网页加载

网页加载需要一段较为明显的时间，此时如果直接定位元素、发送点击等操作会导致异常。 此时可以使用 webdriverwait 操作进行等待。Selenium 预先封装了许多判据供我们直接调用，它们就是 selenium.webdriver.support.expected_conditions

当然，你也可以通过自己定义和函数的方式来设置等待的判据

常见的几种等待的类型：

visibility_of_element_located 当某个元素出现

element_to_be_clickable 当某个元素可以点击

number_of_windows_to_be 当窗口的数目为某个特定值

比如你可以在登入 webVPN 后这样进行等待：如果 id 值为 quick-access-input 的元素出现，那么我们就认为网页充分加载了。

In [17]:
# 用来等待页面加载
from selenium.webdriver.support.ui import WebDriverWait as wdw
from selenium.webdriver.support import expected_conditions as EC

url = By.ID, "quick-access-input"
wdw(d, 5).until(EC.visibility_of_element_located(url))

<selenium.webdriver.remote.webelement.WebElement (session="bff912281120cf5e61415e252b238eb2", element="3fde66c7-6ac5-4873-9020-4d5443936aca")>

这句话的含义是 d 将 wait 直到 url 可见，最大的等待时间是5s，如果超出5s则认为出现问题


### 5.Keys and ActionChain

有的时候你需要模拟按键，那么可以直接利用 selenium.webdriver.common.keys 。

如果只需要按一个键(比方说在搜索框按回车)，那么直接查找到目标搜索框 search ，然后进行 search.send_keys(Key.ENTER)。

但如果需要同时按几个键，则需要通过 action_chain 进行多个操作的联动。

比方说如果 webVPN 的搜索框中原来就有部分内容，你需要先点击搜索框，然后通过 CONTROL(win)/COMMAND(mac)+A 全选，最后按删除键，你可以这样操作：

In [18]:
from selenium.webdriver.common.keys import Keys # 模拟按键
from selenium.webdriver.common.action_chains import ActionChains as AC # action chains

actions = AC(d) # 创建 action_chain 对象

# 可以注意一下这里的 url 为什么加*
# find_element 和 visibility_of_element_located 接受的参数不同哦
actions.move_to_element(d.find_element(*url))
actions.click()
actions.key_down(Keys.COMMAND).send_keys("A").key_up(Keys.COMMAND).send_keys(Keys.DELETE)

# 操作不会立即执行，会在 actions.perform() 的时候一起执行。
actions.perform()

注意，action_chain 将保存所有的 action。直到 actions.perform() 时才依次进行所有的操作


### 6.switch_to.window()

你可以尝试利用 `d.find_element(*url).send_keys("http://info.tsinghua.edu.cn")` 来进入 info 的主页。你会发现这个操作打开了一个新的窗口。我们需要通过 d.window_handles 获取所有窗口的句柄，并找到新窗口，再通过 d.switch_to.window(new_window_handle) 切换到新的窗口继续工作。

In [22]:
# 首先等待第二个窗口出现
wdw(d, 5).until(EC.number_of_windows_to_be(2))

# 然后切换到新的窗口
for window_handle in d.window_handles:
    if window_handle != d.current_window_handle:
        d.switch_to.window(window_handle)

### 7.获取网页源代码

有的时候，通过 requests.get() 获取的是 javascript 代码，而爬虫无法直接执行 javascript 代码获得动态加载的页面。所以需要通过 selenium 登入页面后再获取源码。


### 8.execute_script 执行javascript代码

在按键之外，一个常见的操作是将页面向下拖动以让其加载出更多内容，并判断是否滑到底部。可以用 execute_script() 执行 javascript 代码来完成这个操作

In [23]:
d.execute_script("scrollBy(0, 2500)")
now_height = d.execute_script("return document.documentElement.scrollHeight") # 返回当前的页面高度

最后，在循环中对比 now_height（下滑后的高度）和 prev_height（上一次的高度）即可完成判断页面是否到底的操作

但是注意，如果你把浏览器滑走，可能导致加载速度减慢，新元素加载不出来，浏览器误以为触底。

# 三、实际操作过程的注意事项

*1天速成爬虫，从入门到入狱*

*当你放着爬虫爬一晚上，早上起来刚准备大丰收，结果发现你还没睡着的时候它就不动了*

*你也不知道为什么百度就是不返回正确的页面，一直让你做百度安全检测，一直返回网络异常*

总之，爬虫注意安全性和鲁棒性！！以及必须要要有耐心

## 安全

必须要注意的是，爬敏感数据是违法的，所以千万别还没入门就入狱了。

以及如果你带上了自己的 cookie ，过于频繁操作有被封号的风险。

## 调试

### 1.利用 jupyter notebook 进行小数据量测试

因为爬取的过程中会出现许多许多的错误，所以非常推荐先在 jupyter notebook 上先测试自己的代码。但也要注意notebook的整洁性！

### 2.利用 logging 定位错误

爬取的过程中，要对爬虫的运行状态及产生的异常进行记录，便于后续查看日志，分析运行过程中存在的问题。

这里推荐 Python 自带的 logging 模块。对于已经存在的日志，使用例如 'cat zhihu.log | grep error | less' 就可以查看爬取过程中出现的错误。

可以直接在自己的代码中添加上如下的模块。

In [ ]:
import logging
# 数据格式
fmt = '%(asctime)s.%(msecs)03d [%(levelname)s] %(message)s'
datefmt = '%Y-%m-%d %H:%M:%S'
level = logging.INFO

formatter = logging.Formatter(fmt, datefmt)
logger = logging.getLogger()
logger.setLevel(level)

# 在 youtube.log 中输出信息
file = logging.FileHandler('youtube/youtube.log', encoding='utf-8')
file.setLevel(level)
file.setFormatter(formatter)
logger.addHandler(file)

# console 中输出信息
console = logging.StreamHandler()
console.setLevel(level)
console.setFormatter(formatter)
logger.addHandler(console)

在发生 error 的时候，你可以这么做：
```ipython
except Exception as e:
    logger.error(e)
```

以及在程序的运行过程中也可以添加这样的信息，来方便定位错误。
```ipython
resp = requests.get(url)
logger.info(f"visiting {url}")
logger.info(f"Response {resp.text}")
```


### 3.多层次的错误捕获

爬取的过程中会经常无缘无故出现很多错误，也可能目标网页的页面结构并不完全相同。如果不进行错误捕获可能会导致爬虫频频中止，效率底下。

例如使用 selenium 自动按下 youtube 的 more button，但是不是每一个界面都有 more，如果不 try-except 程序会频繁停止。

```ipython
# 点击 'more' button，但是不是每一个网页都有 more，所以需要 try-except
try:
    btn = driver.find_element(By.XPATH, "//tp-yt-paper-button[@id='expand']")
    time.sleep(5)
    btn.click()
except Exception as e:
    logger.exception(e)
```

又例如百度会经常无缘无故出现 HTTPSConnectionPool 错误，所以需要在整个程序主体之上进行错误的捕获。

实践的过程中我保存了一个记录已经爬取过的url的文件，这样可以断点续爬。


# 四、不同类型的网站对策分析

## 网站常见的反爬措施

1.user-agent: 例如爬虫头 python-requests 。可以用 fake_useragent 作为反制。

2.ip：如果一个 ip 请求频率过高，网站可能就会把这个 ip ban 掉。作为反制，可以设置 time.sleep(n)。如果开始批量出现错误，可以设置 time.sleep(random.uniform(10, 50))中止时间为从10到50的正态分布。也可以购买 ip 池，这样就不用 sleep。

3.cookie：一方面，可能只有加上 cookie 才能访问某些数据，另一方面，网站可能限制 cookie 的访问频率。另外，网站也可能以一定频率更换 cookie 。需要具体问题具体分析。

4.动态渲染：直接请求会得到 javascript 源码而不包含具体信息。可以用 selenium，通过driver.page_source方法获取动态渲染后的代码。

5.验证码：异常访问会触发网页的验证码机制。建议降低访问频率或者人为中止，休息之后再进行爬取。

## 爬取思路

1.先进行**搜索**后批量访问：利用 requests。

先找到搜索 URL 的规律，比如百度贴吧的站内搜索中，判断搜索页数就是通过 URL 最后的 '&pn={page}' 指定的，可以直接对此进行遍历。

再通过 inspect 找到搜索结果网页链接的 html 信息，在遍历过程中通过 beautifulSoup 的 find_all 方法找到这些元素，并将网页的链接解析出来以供访问。

在遍历的过程中，可能需要不断更换 proxy、user-agent 以及 cookie。以及不能忘记合理设置 sleep 的时间。

建议学习如何利用谷歌进行搜索（比如指定必须出现什么关键词、如何指定站点、如何指定时间）。如果站内搜索的结果少，可以在谷歌浏览器中指定 site 以及搜索的时间范围。以及需要学会合理利用网站的高级搜索功能。

BeautifulSoup 详见对应教程

2.静态页面：最为简单的类型，直接利用 requests。可以直接通过 url 请求获取想要的信息。注意可能需要提供 user-agent 以及 cookie，以及不能忘记设置 sleep。

3.动态页面：利用 selenium。事实上，selenium 可以逃脱大多数的反爬机制，但相应牺牲了爬取的效率。例如多数社交媒体网站，需要不断向下滑动以不断获取新的帖子。这种情况就需要利用 selenium'scrollBy' 向下滑动，利用 page_source 获取网页代码或者直接用 beautifulSoup 进行数据分析。


## 最后

学习爬虫，可以多试试不同类型的网站，最后可以发现很多共通之处，很多地方的操作都是非常类似的。总之，需要有耐心、不断试错。一开始可能觉得很麻烦，到后面会觉得也还好。

